## Analyse network data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os

rows_limit = 5527409

In [40]:
# Open attack_1.csv

df_attack1 = pd.read_csv('/home/johan/Desktop/PDSSC/Projet/data/dataset/Network datatset/csv/attack_1.csv', delimiter=',', header=0, nrows=rows_limit) # 5527409 entries

In [41]:
# Change the name of the columns by removing the spaces
df_attack1.columns = df_attack1.columns.str.replace(' ', '')

In [43]:
# Open normal.csv 

df_normal = pd.read_csv('/home/johan/Desktop/PDSSC/Projet/data/dataset/Network datatset/csv/normal.csv', delimiter=',', header=0, nrows=rows_limit)

In [44]:
# Remove duplicates from both dataframes

df_attack1 = df_attack1.drop_duplicates()
df_normal = df_normal.drop_duplicates()

In [45]:
def get_columns_info(column_name):
    print('Column name: ', column_name)
    print('Number of unique values in attack_1: ', df_attack1[column_name].nunique())
    print('Number of unique values in normal: ', df_normal[column_name].nunique())
    print('Number of missing values in attack_1: ', df_attack1[column_name].isnull().sum())
    print('Number of missing values in normal: ', df_normal[column_name].isnull().sum())
    print('Number of values in attack_1: ', df_attack1[column_name].count())
    print('Number of values in normal: ', df_normal[column_name].count())
    print('Values in attack_1: ', df_attack1[column_name].unique())
    print('Values in normal: ', df_normal[column_name].unique())

# Début Rapport d'analyse

problème avec ces colonne : 'ip_s', 'ip_d', 'sport', 'dport', 'flags', 'n_pkt_src', 'n_pkt_dst', 'modbus_fn', 'modbus_response'
raison : missing some values

problème avec modbus_fn, lot of missing value (153121 for attack1), useless column ? Même chose avec modbus_response : Colonne lié au proto modbus et donc pas toute les colonne

La colonne proto contient les protocoles ARP ICMP Modbus TCP pour attack mais seulement Modbus et TCP pour le cas normal

Problème de typage : 'sport', 'dport', 'flags', 'n_pkt_src', 'n_pkt_dst'

Certaine dates sont partagé pour plusieurs entrées (ex : 2021-04-09 18:42:46.997219 a 4 entrés)

Il y a des cas noté normaux dans les csv attack, besoin du csv normal ?


In [46]:
df_attack1.columns

Index(['Time', 'mac_s', 'mac_d', 'ip_s', 'ip_d', 'sport', 'dport', 'proto',
       'flags', 'size', 'modbus_fn', 'n_pkt_src', 'n_pkt_dst',
       'modbus_response', 'label_n', 'label'],
      dtype='object')

In [80]:
get_columns_info('Time')

Column name:  Time
Number of unique values in attack_1:  5241591
Number of unique values in normal:  5082414
Number of missing values in attack_1:  0
Number of missing values in normal:  0
Number of values in attack_1:  5526894
Number of values in normal:  5527409
Values in attack_1:  ['2021-04-09 18:23:28.385003' '2021-04-09 18:23:28.385005'
 '2021-04-09 18:23:28.385006' ... '2021-04-09 19:03:47.661288'
 '2021-04-09 19:03:47.661290' '2021-04-09 19:03:47.661291']
Values in normal:  ['2021-04-09 11:30:52.716203' '2021-04-09 11:30:52.716499'
 '2021-04-09 11:30:52.717334' ... '2021-04-09 12:11:32.090901'
 '2021-04-09 12:11:32.091494' '2021-04-09 12:11:32.091824']


In [76]:
# Remove NaN values from the column list

column_NaN = ['mac_s', 'mac_d', 'ip_s', 'ip_d', 'sport', 'dport', 'flags', 'proto', 'size', 'n_pkt_src', 'n_pkt_dst','label']

df_attack1 = df_attack1.dropna(subset=column_NaN)

In [77]:
# Convert the columns to the right type

column_type = ['sport', 'dport', 'flags', 'n_pkt_src', 'n_pkt_dst']

for column in column_type:
    df_attack1[column] = df_attack1[column].astype('int64')

In [87]:
# Save the cleaned dataframes

df_attack1.to_csv('/home/johan/Desktop/PDSSC/Projet/data/dataset/Network datatset/clean_csv/attack_1_cleaned.csv', index=False)

In [88]:
# KNN algorithm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load the cleaned dataframes

df_attack1 = pd.read_csv('/home/johan/Desktop/PDSSC/Projet/data/dataset/Network datatset/clean_csv/attack_1_cleaned.csv', delimiter=',', header=0)

In [ ]:
# train_test_split

X = df_attack1[['sport', 'dport', 'flags', 'n_pkt_src', 'n_pkt_dst']]